In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
languoid = pd.read_csv("../data/VG_MKT_suppl/DataStageI/languoid.csv")

In [38]:
languoid[languoid["id"]=="pawa1255"]

,id,family_id,parent_id,name,bookkeeping,level,latitude,longitude,iso639P3code,description,markup_description,child_family_count,child_language_count,child_dialect_count,country_ids
17170,pawa1255,NaN,NaN,Pawaia,False,language,-6.88021,145.081,pwa,NaN,NaN,0,0,2,PG


In [47]:
df

,id,name,family_id,top-level family,macroarea,latitude,longitude,iso639-3
0,musk1252,Muskogean,NaN,NaN,NaN,NaN,NaN,NaN
1,maba1274,Maban,NaN,NaN,NaN,NaN,NaN,NaN
2,guri1248,Guriaso,NaN,NaN,Papunesia,-3.57180,141.597,grx
3,pawa1255,Pawaia,NaN,NaN,Papunesia,-6.88021,145.081,pwa
4,bora1262,Boran,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26411,amur1242,Amur-West Sakhalin Nivkh,nivk1234,Nivkh,Eurasia,NaN,NaN,NaN
26412,nort2750,North Sakhalin Nivkh,nivk1234,Nivkh,Eurasia,NaN,NaN,NaN
26413,cent2402,Central Sakhalin Nivkh,nivk1234,Nivkh,Eurasia,NaN,NaN,NaN
26414,sout3327,South Sakhalin Nivkh,nivk1234,Nivkh,Eurasia,NaN,NaN,NaN


In [41]:
df = pd.read_csv("../data/languages/languages_family_area.csv")

In [112]:
id2name = dict(zip(df["id"], df["name"]))

In [43]:
id2fam = dict(zip(df["family_id"], df["top-level family"]))
                  

In [49]:
language_dict= dict()
for glottocode, family, name, latitude, longitude in zip(df["id"], df["top-level family"], df["name"], df["latitude"], df["longitude"]):
    language_dict[glottocode]={"glottolog.family":family,
                              "glottolog.name":name,
                              "latitude":latitude,
                              "longitude":longitude}



In [40]:
fam_map = {'indo1319': 'Indo-European',
           'turk1311': 'Turkic',
           'ural1272': 'Uralic',
           'basq1248': 'Basque'}

In [57]:
languoid.head(2)

,id,family_id,parent_id,name,bookkeeping,level,latitude,longitude,iso639P3code,description,markup_description,child_family_count,child_language_count,child_dialect_count,country_ids
0,3adt1234,afro1255,nort3292,3Ad-Tekles,False,dialect,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
1,aala1237,aust1307,ramo1244,Aalawa,False,dialect,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN


In [73]:
languoid["parent_id"].dropna()

0        nort3292
1        ramo1244
2        nort2920
3        ahkk1235
4        aari1238
           ...   
24696    guaj1255
24697    omie1241
24698    shal1242
24699    nucl1757
24700    komi1269
Name: parent_id, Length: 24272, dtype: object

In [92]:
languoid["parent_id_len"]= languoid["parent_id"].str.len()

In [95]:
languoid_parent = languoid[["id", "parent_id", "parent_id_len"]]

In [97]:
parent_df = languoid_parent[languoid_parent["parent_id_len"]==8]

In [55]:
with open("../data/languages/languageSubfamilies_glottolog.json") as f:
    sub_families = json.load(f)

In [59]:
sub_family_map = sub_families

In [98]:
parent_map = {x: y for x, y in dict(zip(parent_df["id"], parent_df["parent_id"])).items()}

In [99]:
len(parent_map)

24272

In [101]:
fam_map

{'indo1319': 'Indo-European',
 'turk1311': 'Turkic',
 'ural1272': 'Uralic',
 'basq1248': 'Basque'}

In [76]:
parent_map["kanu1280"]

'more1255'

In [63]:
len(sub_families)

4425

In [62]:
len(sub_family_map)

4425

In [68]:
len(parent_map)

24701

In [105]:
def get_parent(l, parent_map, sub_families):
    parent = parent_map[l]
    if parent not in sub_families:
        parent = get_parent(parent, parent_map, sub_families)
    return(parent)

In [39]:
langs= languoid["id"].tolist()

In [117]:
sub_family_map["alab1247"]

'Alabaman-Koasati'

In [147]:
# original (refer to the Gast, V. & Koptjevskaja-Tamm, M. (2022).)
for l in language_dict:
    print(l)
    if l not in sub_families and l not in fam_map:
        try:
            parent = get_parent(l, parent_map, sub_families)
            language_dict[l]['glottolog.branch'] = sub_family_map[parent]
        except Exception:
            continue
    # this is the case of Hungarian
    if l in sub_families and l not in fam_map:
        language_dict[l]['glottolog.branch'] = id2name[l]
    # this is the case of Basque and Indo-European
    if l in fam_map:
        language_dict[l]['glottolog.branch'] = id2name[l]

musk1252
maba1274
guri1248
pawa1255
bora1262
guam1236
rash1249
dama1272
puqu1242
kusu1250
mirn1241
kuli1252
vile1241
tuni1252
sent1261
arut1244
yetf1238
hata1242
araf1243
movi1243
tamb1257
buna1274
teme1251
cuit1236
muni1258
sout1293
taia1239
chiq1253
chim1301
klam1254
yuru1243
uruc1242
west2604
nduu1242
anga1289
coah1252
pomo1273
mixe1287
aust1307
cayu1262
east1459
yeni1252
toto1251
yama1264
arti1236
kunz1244
mail1249
shom1245
iber1250
nucl1710
boro1281
bira1253
lule1238
hara1260
yele1255
miwo1274
pauw1244
pele1245
left1242
lowe1437
mang1423
hurr1239
krua1234
iton1250
guai1237
tonk1249
araw1281
piaw1238
chuk1271
kond1302
usku1243
guah1252
maku1246
timo1261
iroq1247
jiva1245
jarr1235
haid1248
atak1252
nort1547
adai1235
lavu1241
east2503
molo1262
sout1516
tuxa1239
wara1303
pidg1258
ainu1252
yang1287
beot1247
abkh1242
jica1245
saus1247
sign1238
khoe1240
misu1242
kimk1238
puri1261
fasu1242
kaya1327
sout2772
coos1248
buru1296
mure1235
taik1256
zamu1243
cofa1242
algi1248
coto1248
skoo1245
i

In [ ]:
count =1
for lang, d in language_dict.items():
    if "glottolog.branch" in d:
        print(lang, d["glottolog.name"], d["glottolog.branch"])

In [122]:
lang_df_new = pd.DataFrame.from_dict(language_dict, orient="index")

In [123]:
lang_df_new

,glottolog.family,glottolog.name,latitude,longitude,glottolog.branch
musk1252,NaN,Muskogean,NaN,NaN,musk1252
maba1274,NaN,Maban,NaN,NaN,maba1274
guri1248,NaN,Guriaso,-3.57180,141.597,NaN
pawa1255,NaN,Pawaia,-6.88021,145.081,NaN
bora1262,NaN,Boran,NaN,NaN,NaN
...,...,...,...,...,...
amur1242,Nivkh,Amur-West Sakhalin Nivkh,NaN,NaN,NaN
nort2750,Nivkh,North Sakhalin Nivkh,NaN,NaN,NaN
cent2402,Nivkh,Central Sakhalin Nivkh,NaN,NaN,NaN
sout3327,Nivkh,South Sakhalin Nivkh,NaN,NaN,NaN


In [126]:
branch_dict = dict(zip(lang_df_new.index, lang_df_new["glottolog.branch"]))

In [124]:
languagedf = pd.read_csv("../data/languages/languages_dataframe.csv")

In [127]:
languagedf["branch"] = languagedf["id"].apply(lambda x: branch_dict.get(x, np.NaN))

In [129]:
languagedf["branch"].value_counts()

branch
Japan-Taiwan Japanese           127
Macro-English                    80
Eastern-Western Armenian         70
Malayo-Polynesian                67
Middle-Modern Sinitic            58
                               ... 
Nuclear Oti-Volta Occidental      1
Northwest Oti-Volta               1
Southeastern Chinantec            1
Mossi-Farefare                    1
North Germanic                    1
Name: count, Length: 4431, dtype: int64

In [35]:
parent_map = dict(zip(languoid["id"], languoid["parent_id"]))

In [36]:
parent_map

{'3adt1234': 'nort3292',
 'aala1237': 'ramo1244',
 'aant1238': 'nort2920',
 'aari1238': 'ahkk1235',
 'aari1239': 'aari1238',
 'aari1240': 'book1242',
 'aasa1238': 'unun9872',
 'aasd1234': 'ferr1240',
 'aata1238': 'sout2943',
 'abaa1238': 'nort3305',
 'abab1239': 'beja1238',
 'abab1240': 'narr1281',
 'abad1240': 'adyg1241',
 'abad1241': 'west2850',
 'abag1245': 'unun9976',
 'abai1239': 'lund1271',
 'abai1240': 'pait1248',
 'abai1241': 'muru1275',
 'abak1242': 'anaa1238',
 'abak1243': 'ilon1239',
 'abal1238': 'yele1255',
 'abam1239': 'wipi1242',
 'aban1242': 'aban1244',
 'aban1243': 'east2551',
 'aban1244': 'bako1254',
 'abar1238': 'sout3152',
 'abau1245': 'sepi1257',
 'abaw1238': 'gupa1248',
 'abay1238': 'agwa1247',
 'abaz1241': 'abkh1243',
 'abbe1238': 'abee1242',
 'abbr1238': 'sout3309',
 'abda1238': 'soqo1240',
 'abed1238': 'lugb1240',
 'abee1242': 'agne1238',
 'abee1243': 'anyi1245',
 'abel1234': 'samb1319',
 'aben1247': 'garo1247',
 'aben1248': 'garo1247',
 'aben1249': 'abel1234',


In [6]:
# 01.prepare data

In [ ]:
subfam_map = b
famil

In [4]:
dataframe = pd.read_csv("../data/VG_MKT_suppl/DataStageIII/dataframe.tsv", sep="\t")

In [5]:
dataframe

,glottocode,glottolog.family,glottolog.name,latitude,longitude,asjp.id,glottolog.branch,wals.family,wals.genus,europe,...,WHATxWHO,WHATxWHY,WHEATxWHITE,WHERExWHITHER.oWHERE.TOc,WHICHxWHO,WIDOWxWIDOWER,WIFExWOMAN,WORK.oACTIVITYcxWORK.oLABOURc,YOUNGxYOUNG.MAN,YOUNG.MANxYOUNG.WOMAN
vene1258,vene1258,Indo-European,Venetian,45.0000,12.214167,NaN,Italic,NaN,NaN,True,...,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN
hind1269,hind1269,Indo-European,Hindi,25.0000,77.000000,HINDI,Indo-Iranian,IE,INDIC,False,...,f,f,f,t,f,f,f,f,f,f
roma1328,roma1328,Indo-European,Romagnol,44.2349,11.718900,ROMAGNOL,Italic,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gali1258,gali1258,Indo-European,Galician,42.2446,-7.534300,GALICIAN,Italic,IE,ROMANCE,True,...,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN
osse1243,osse1243,Indo-European,Ossetian,42.9800,44.610000,IRON_OSSETIAN,Indo-Iranian,IE,IRANIAN,False,...,f,f,f,f,f,f,t,f,t,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
turk1304,turk1304,Turkic,Turkmen,37.0872,59.175700,TURKMEN,Common Turkic,Alt,TURKIC,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gaga1249,gaga1249,Turkic,Gagauz,44.5988,28.027700,GAGAUZ,Common Turkic,Alt,TURKIC,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crim1257,crim1257,Turkic,Crimean Tatar,45.0000,34.080000,CRIMEAN_TATAR,Common Turkic,Alt,TURKIC,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tuvi1240,tuvi1240,Turkic,Tuvinian,51.7000,94.366670,TUVAN,Common Turkic,Alt,TURKIC,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
languagedf.to_csv("../data/languages/languages_dataframe.csv",index=False)

In [131]:
languagedf

,Unnamed: 0,id,parent,name,level,hid,iso639_3,latitude,longitude,macroareas,countries,timespan,branch
0,0,abin1243,NaN,Abinomn,language,bsa,bsa,-2.922810,138.891000,['Papunesia'],['Indonesia'],NaN,NaN
1,1,abis1238,NaN,Aewa,language,ash,ash,-1.284096,-75.084405,['South America'],['Peru'],NaN,NaN
2,2,abkh1242,NaN,Abkhaz-Adyge,family,NaN,NaN,NaN,NaN,[],[],NaN,NaN
3,3,abkh1243,abkh1242,Abkhaz-Abaza,family,NaN,NaN,NaN,NaN,[],[],NaN,Abkhaz-Abaza
4,4,abaz1241,abkh1243,Abaza,language,abq,abq,44.250000,42.000000,['Eurasia'],"['Russian Federation', 'Turkey']",NaN,Abkhaz-Abaza
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26411,26411,pint1253,iqui1243,Pintuyacu,dialect,NaN,NaN,NaN,NaN,['South America'],[],NaN,Cahuarano-Iquito
26412,26412,zapa1252,zapa1251,Zaparo-Abishira,family,NaN,NaN,NaN,NaN,[],[],NaN,Zaparo-Abishira
26413,26413,aush1242,zapa1252,Aushiri,language,avs,avs,-3.319600,-74.541100,['South America'],['Peru'],NaN,Zaparo-Abishira
26414,26414,zapa1253,zapa1252,Záparo,language,zro,zro,-1.998710,-76.364000,['South America'],"['Ecuador', 'Peru']",NaN,Zaparo-Abishira


In [146]:
len(parent_map)

24272

In [133]:
df_data = pd.read_csv("../data/stage1/languages_coords.csv")

In [143]:
df_data["parent_id"] = df_data["id"].apply(lambda x: parent_map.get(x, np.NaN))

In [ ]:
branch_dict

In [135]:
df_data["branch"]=df_data["id"].apply(lambda x: branch_dict.get(x, np.NaN))

In [137]:
df_data.branch.value_counts()

branch
Northern Core Zapotec    8
Nuru                     7
Ecuadorian Quechua B     6
Central Alor             5
Central Core Zapotec     5
                        ..
Uighur-Ili               1
Crimeaic                 1
East Kipchak             1
Yenisey Turkic           1
Ngombe-Genja             1
Name: count, Length: 1017, dtype: int64

In [138]:
len(df_data)

1561

In [144]:
df_data

,id,name,family_id,top-level family,macroarea,latitude,longitude,iso639-3,branch,parent_id
0,pawa1255,Pawaia,NaN,NaN,Papunesia,-6.880210,145.081000,pwa,NaN,NaN
1,yetf1238,Yetfa,NaN,NaN,Papunesia,-4.058840,140.653000,yet,NaN,NaN
2,movi1243,Movima,NaN,NaN,South America,-13.811500,-65.634800,mzp,NaN,NaN
3,cayu1262,Cayubaba,NaN,NaN,South America,-13.248200,-65.482200,cyb,NaN,NaN
4,yama1264,Yámana,NaN,NaN,South America,-55.274800,-68.264800,yag,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1556,sili1247,Siliput,nucl1708,Nuclear Torricelli,Papunesia,-3.608380,142.397000,mkc,Nuclear Maimai,nucl1590
1557,kama1367,Kamasau,nucl1708,Nuclear Torricelli,Papunesia,-3.850300,143.842000,kms,Elepi-Kamasau-Marienberg,elep1241
1558,nort2971,Enlhet Norte,leng1261,Lengua-Mascoy,South America,-22.540000,-58.980000,enl,Lengua,leng1262
1559,sout2989,Enxet Sur,leng1261,Lengua-Mascoy,South America,-23.427556,-57.945626,enx,Lengua,leng1262


In [145]:
df_data.to_csv("../data/stage1/languages_info.csv", index=False)